In [1]:
import cantera as ct

In [ ]:
#first, edit the file so it doesn't overwrite the smiles
path = './pfca_final.yaml'   #load in thermo file
with open(path, 'r') as f: 
    lines = f.readlines()

SMILES_indices = []
for ind, line in enumerate(lines): 
    if "SMILES" in line: 
        SMILES_indices.append(ind)

indices_to_remove = []
#now look at the next three lines after these SMILE lines
for ind in SMILES_indices: 
    for x in range(1,5):
        next_ind = ind + x
        if "note:" in lines[next_ind]: 
            #we will not be saving this line
            indices_to_remove.append(next_ind)


In [37]:
for x in range(1,5):
    print(x)

1
2
3
4


In [10]:
if "F" in species.composition.keys():
    print('yeah')

yeah


In [14]:
gas = ct.Solution(path)     

fluorinated_species = []

for species in gas.species():
    if "F" in species.composition.keys():
        fluorinated_species.append(species)



In [32]:
fluorinated_species[30].from_yaml()

TypeError: from_yaml() takes exactly 1 positional argument (0 given)

In [13]:
final_string = ''

for ind, spc in enumerate(fluorinated_species): 
    #get the label
    label = spc.name

    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
